In [1]:
#모듈
import math
import pandas as pd
import numpy as np

try:
    import cufflinks as cf
except:
    !pip install cufflinks
    import cufflinks as cf

cf.go_offline()

from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor,BaggingRegressor
from sklearn.neural_network import MLPRegressor

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [2]:
# 데이터 다루기
pd.set_option("Display.max_columns",200)
import re
import time

# 데이터 시각화
import seaborn as sns

# 사용할 머신러닝 모델들 - RandomForest, AdaBoost, GradientBoost, ExtraTrees
import sklearn
from sklearn.metrics import log_loss
from sklearn.svm import SVC

In [3]:
#명목형 변수 더미화
def dummify_category_cols(df,cols):
    dummies = []
    new_df = df.copy()

    for idx in range(len(cols)):
        new_df = new_df.join(pd.get_dummies(total[cols[idx]], prefix = cols[idx]))
        new_df = new_df.drop(cols[idx], axis =1)
    return new_df

In [4]:
#연속형 변수 범주화
def dummify_number_cols(df, cols, nums) :
    new_df = df.copy() 
    
    for col, num in zip(cols, nums) :
        new_df[col] =new_df[col].map(lambda x: int(x)//num * num)
    return new_df


In [5]:
#명목형 변수의 연속형 변환 - 변수의 차원을 낮추기 위하여
def num_category(df, col, chg) :
    new_df = df.copy()
    set = {}
    
    for idx in range(len(chg)) :
        set[chg[idx]] = idx
        
    new_df[col] = new_df[col].map(set)
    
    return new_df        

In [6]:
#숫자형 변수 일정 이상은 묶어버리기
def number_group_cols(df, cols, nums) :
    new_df = df.copy() 
    
    for col, num in zip(cols, nums) :
        new_df[col] =new_df[col].map(lambda x: num if x> num else x )
    return new_df


In [45]:
#RMS 구하기
def RMSE(y_true,y_pred):
    
    squaredErrors = (y_pred - y_true)**2
    MSE = squaredErrors.mean()
    
    return math.sqrt(MSE)

In [46]:
#본격적으로 시작

In [47]:
training = pd.read_csv("house_price.csv")
test = pd.read_csv("test.csv")


In [48]:
training = training[training.GrLivArea <= 5600]
#training = training[training["1stFlrSF"] <= 4600]
#training = training[training.TotalBsmtSF < 6000]
#training = training[training.MasVnrArea <= 1600]

In [49]:
total = pd.concat([training, test]).reset_index()
del total["index"]

In [50]:
total.head(2)

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,BsmtFinType2,BsmtFullBath,BsmtHalfBath,BsmtQual,BsmtUnfSF,CentralAir,Condition1,Condition2,Electrical,EnclosedPorch,ExterCond,ExterQual,Exterior1st,Exterior2nd,Fence,FireplaceQu,Fireplaces,Foundation,FullBath,Functional,GarageArea,GarageCars,GarageCond,GarageFinish,GarageQual,GarageType,GarageYrBlt,GrLivArea,HalfBath,Heating,HeatingQC,HouseStyle,Id,KitchenAbvGr,KitchenQual,LandContour,LandSlope,LotArea,LotConfig,LotFrontage,LotShape,LowQualFinSF,MSSubClass,MSZoning,MasVnrArea,MasVnrType,MiscFeature,MiscVal,MoSold,Neighborhood,OpenPorchSF,OverallCond,OverallQual,PavedDrive,PoolArea,PoolQC,RoofMatl,RoofStyle,SaleCondition,SalePrice,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,NaN,3,1Fam,TA,No,706.0,0.0,GLQ,Unf,1.0,0.0,Gd,150.0,Y,Norm,Norm,SBrkr,0,TA,Gd,VinylSd,VinylSd,NaN,NaN,0,PConc,2,Typ,548.0,2.0,TA,RFn,TA,Attchd,2003.0,1710,1,GasA,Ex,2Story,1,1,Gd,Lvl,Gtl,8450,Inside,65.0,Reg,0,60,RL,196.0,BrkFace,NaN,0,2,CollgCr,61,5,7,Y,0,NaN,CompShg,Gable,Normal,208500.0,WD,0,Pave,8,856.0,AllPub,0,2003,2003,2008
1,1262,0,0,NaN,3,1Fam,TA,Gd,978.0,0.0,ALQ,Unf,0.0,1.0,Gd,284.0,Y,Feedr,Norm,SBrkr,0,TA,TA,MetalSd,MetalSd,NaN,TA,1,CBlock,2,Typ,460.0,2.0,TA,RFn,TA,Attchd,1976.0,1262,0,GasA,Ex,1Story,2,1,TA,Lvl,Gtl,9600,FR2,80.0,Reg,0,20,RL,0.0,None,NaN,0,5,Veenker,0,8,6,Y,0,NaN,CompShg,Gable,Normal,181500.0,WD,0,Pave,6,1262.0,AllPub,298,1976,1976,2007


In [51]:
total.columns

Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'Alley', 'BedroomAbvGr',
       'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtFinType1', 'BsmtFinType2', 'BsmtFullBath', 'BsmtHalfBath',
       'BsmtQual', 'BsmtUnfSF', 'CentralAir', 'Condition1', 'Condition2',
       'Electrical', 'EnclosedPorch', 'ExterCond', 'ExterQual', 'Exterior1st',
       'Exterior2nd', 'Fence', 'FireplaceQu', 'Fireplaces', 'Foundation',
       'FullBath', 'Functional', 'GarageArea', 'GarageCars', 'GarageCond',
       'GarageFinish', 'GarageQual', 'GarageType', 'GarageYrBlt', 'GrLivArea',
       'HalfBath', 'Heating', 'HeatingQC', 'HouseStyle', 'Id', 'KitchenAbvGr',
       'KitchenQual', 'LandContour', 'LandSlope', 'LotArea', 'LotConfig',
       'LotFrontage', 'LotShape', 'LowQualFinSF', 'MSSubClass', 'MSZoning',
       'MasVnrArea', 'MasVnrType', 'MiscFeature', 'MiscVal', 'MoSold',
       'Neighborhood', 'OpenPorchSF', 'OverallCond', 'OverallQual',
       'PavedDrive', 'PoolArea', 'Po

In [52]:
#선택된 변수
category_selected = ["MSSubClass", "MSZoning", "Street" ,"LandContour", "LotConfig", 
            "Neighborhood", "BldgType", "HouseStyle", "OverallQual", 
            "MasVnrType", "ExterQual", "ExterCond",
            "Foundation","BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "HeatingQC", "CentralAir",
            "Electrical", "BsmtFullBath", "FullBath", "HalfBath", "KitchenQual",
            "TotRmsAbvGrd", "Functional", "Fireplaces", "FireplaceQu", "GarageType",
            "GarageCars", "PavedDrive", "SaleCondition", "GarageFinish"]

extra_selected = [ "GarageCars", "Electrical" ,"OverallQual","OverallCond","ExterQual", "ExterCond",  "BsmtQual",
                  "BsmtCond", "BsmtExposure", "BsmtFinType1", "HeatingQC","KitchenQual","FireplaceQu", "Functional"]

number_selected = ["GarageYrBlt","YearBuilt", "YearRemodAdd", "BsmtFinSF1", "BsmtUnfSF", "TotalBsmtSF", 
                   "1stFlrSF", "2ndFlrSF", "GrLivArea", 'GarageArea', "LotArea", "MasVnrArea", "WoodDeckSF", "OpenPorchSF"]

category_selected = [x for x in category_selected if x not in extra_selected]

total_selected = category_selected + number_selected + extra_selected

selected_with_SalePrice = total_selected +["SalePrice"]

In [53]:
#선택된 변수
category_selected = ['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'Alley', 'BedroomAbvGr',
       'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtFinType1', 'BsmtFinType2', 'BsmtFullBath', 'BsmtHalfBath',
       'BsmtQual', 'BsmtUnfSF', 'CentralAir', 'Condition1', 'Condition2',
       'Electrical', 'EnclosedPorch', 'ExterCond', 'ExterQual', 'Exterior1st',
       'Exterior2nd', 'Fence', 'FireplaceQu', 'Fireplaces', 'Foundation',
       'FullBath', 'Functional', 'GarageArea', 'GarageCars', 'GarageCond',
       'GarageFinish', 'GarageQual', 'GarageType', 'GarageYrBlt', 'GrLivArea',
       'HalfBath', 'Heating', 'HeatingQC', 'HouseStyle', 'Id', 'KitchenAbvGr',
       'KitchenQual', 'LandContour', 'LandSlope', 'LotArea', 'LotConfig',
       'LotFrontage', 'LotShape', 'LowQualFinSF', 'MSSubClass', 'MSZoning',
       'MasVnrArea', 'MasVnrType', 'MiscFeature', 'MiscVal', 'MoSold',
       'Neighborhood', 'OpenPorchSF', 'OverallCond', 'OverallQual',
       'PavedDrive', 'PoolArea', 'PoolQC', 'RoofMatl', 'RoofStyle',
       'SaleCondition', 'SaleType', 'ScreenPorch', 'Street',
       'TotRmsAbvGrd', 'TotalBsmtSF', 'Utilities', 'WoodDeckSF', 'YearBuilt',
       'YearRemodAdd', 'YrSold']

extra_selected = [ "GarageCars", "Electrical" ,"OverallQual","OverallCond","ExterQual", "ExterCond",  "BsmtQual",
                  "BsmtCond", "BsmtExposure", "BsmtFinType1", "HeatingQC","KitchenQual","FireplaceQu", "Functional"]

number_selected =['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea','GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']
category_selected = [x for x in category_selected if x not in number_selected]

category_selected = [x for x in category_selected if x not in extra_selected]

total_selected = category_selected + number_selected + extra_selected

selected_with_SalePrice = total_selected +["SalePrice"]

In [54]:
new_total = total[selected_with_SalePrice].copy()
new_total.head(2)

,Alley,BedroomAbvGr,BldgType,BsmtFinType2,BsmtFullBath,BsmtHalfBath,CentralAir,Condition1,Condition2,Exterior1st,Exterior2nd,Fence,Fireplaces,Foundation,FullBath,GarageCond,GarageFinish,GarageQual,GarageType,GarageYrBlt,HalfBath,Heating,HouseStyle,Id,KitchenAbvGr,LandContour,LandSlope,LotConfig,LotShape,MSSubClass,MSZoning,MasVnrType,MiscFeature,MoSold,Neighborhood,PavedDrive,PoolQC,RoofMatl,RoofStyle,SaleCondition,SaleType,Street,TotRmsAbvGrd,Utilities,YrSold,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,GarageCars,Electrical,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,KitchenQual,FireplaceQu,Functional,SalePrice
0,NaN,3,1Fam,Unf,1.0,0.0,Y,Norm,Norm,VinylSd,VinylSd,NaN,0,PConc,2,TA,RFn,TA,Attchd,2003.0,1,GasA,2Story,1,1,Lvl,Gtl,Inside,Reg,60,RL,BrkFace,NaN,2,CollgCr,Y,NaN,CompShg,Gable,Normal,WD,Pave,8,AllPub,2008,65.0,8450,2003,2003,196.0,706.0,0.0,150.0,856.0,856,854,0,1710,548.0,0,61,0,0,0,0,0,2.0,SBrkr,7,5,Gd,TA,Gd,TA,No,GLQ,Ex,Gd,NaN,Typ,208500.0
1,NaN,3,1Fam,Unf,0.0,1.0,Y,Feedr,Norm,MetalSd,MetalSd,NaN,1,CBlock,2,TA,RFn,TA,Attchd,1976.0,0,GasA,1Story,2,1,Lvl,Gtl,FR2,Reg,20,RL,None,NaN,5,Veenker,Y,NaN,CompShg,Gable,Normal,WD,Pave,6,AllPub,2007,80.0,9600,1976,1976,0.0,978.0,0.0,284.0,1262.0,1262,0,0,1262,460.0,298,0,0,0,0,0,0,2.0,SBrkr,6,8,TA,TA,Gd,TA,Gd,ALQ,Ex,TA,TA,Typ,181500.0


In [55]:
#임시 전처리

In [56]:
new_total.head()

,Alley,BedroomAbvGr,BldgType,BsmtFinType2,BsmtFullBath,BsmtHalfBath,CentralAir,Condition1,Condition2,Exterior1st,Exterior2nd,Fence,Fireplaces,Foundation,FullBath,GarageCond,GarageFinish,GarageQual,GarageType,GarageYrBlt,HalfBath,Heating,HouseStyle,Id,KitchenAbvGr,LandContour,LandSlope,LotConfig,LotShape,MSSubClass,MSZoning,MasVnrType,MiscFeature,MoSold,Neighborhood,PavedDrive,PoolQC,RoofMatl,RoofStyle,SaleCondition,SaleType,Street,TotRmsAbvGrd,Utilities,YrSold,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,GarageCars,Electrical,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,KitchenQual,FireplaceQu,Functional,SalePrice
0,NaN,3,1Fam,Unf,1.0,0.0,Y,Norm,Norm,VinylSd,VinylSd,NaN,0,PConc,2,TA,RFn,TA,Attchd,2003.0,1,GasA,2Story,1,1,Lvl,Gtl,Inside,Reg,60,RL,BrkFace,NaN,2,CollgCr,Y,NaN,CompShg,Gable,Normal,WD,Pave,8,AllPub,2008,65.0,8450,2003,2003,196.0,706.0,0.0,150.0,856.0,856,854,0,1710,548.0,0,61,0,0,0,0,0,2.0,SBrkr,7,5,Gd,TA,Gd,TA,No,GLQ,Ex,Gd,NaN,Typ,208500.0
1,NaN,3,1Fam,Unf,0.0,1.0,Y,Feedr,Norm,MetalSd,MetalSd,NaN,1,CBlock,2,TA,RFn,TA,Attchd,1976.0,0,GasA,1Story,2,1,Lvl,Gtl,FR2,Reg,20,RL,None,NaN,5,Veenker,Y,NaN,CompShg,Gable,Normal,WD,Pave,6,AllPub,2007,80.0,9600,1976,1976,0.0,978.0,0.0,284.0,1262.0,1262,0,0,1262,460.0,298,0,0,0,0,0,0,2.0,SBrkr,6,8,TA,TA,Gd,TA,Gd,ALQ,Ex,TA,TA,Typ,181500.0
2,NaN,3,1Fam,Unf,1.0,0.0,Y,Norm,Norm,VinylSd,VinylSd,NaN,1,PConc,2,TA,RFn,TA,Attchd,2001.0,1,GasA,2Story,3,1,Lvl,Gtl,Inside,IR1,60,RL,BrkFace,NaN,9,CollgCr,Y,NaN,CompShg,Gable,Normal,WD,Pave,6,AllPub,2008,68.0,11250,2001,2002,162.0,486.0,0.0,434.0,920.0,920,866,0,1786,608.0,0,42,0,0,0,0,0,2.0,SBrkr,7,5,Gd,TA,Gd,TA,Mn,GLQ,Ex,Gd,TA,Typ,223500.0
3,NaN,3,1Fam,Unf,1.0,0.0,Y,Norm,Norm,Wd Sdng,Wd Shng,NaN,1,BrkTil,1,TA,Unf,TA,Detchd,1998.0,0,GasA,2Story,4,1,Lvl,Gtl,Corner,IR1,70,RL,None,NaN,2,Crawfor,Y,NaN,CompShg,Gable,Abnorml,WD,Pave,7,AllPub,2006,60.0,9550,1915,1970,0.0,216.0,0.0,540.0,756.0,961,756,0,1717,642.0,0,35,272,0,0,0,0,3.0,SBrkr,7,5,TA,TA,TA,Gd,No,ALQ,Gd,Gd,Gd,Typ,140000.0
4,NaN,4,1Fam,Unf,1.0,0.0,Y,Norm,Norm,VinylSd,VinylSd,NaN,1,PConc,2,TA,RFn,TA,Attchd,2000.0,1,GasA,2Story,5,1,Lvl,Gtl,FR2,IR1,60,RL,BrkFace,NaN,12,NoRidge,Y,NaN,CompShg,Gable,Normal,WD,Pave,9,AllPub,2008,84.0,14260,2000,2000,350.0,655.0,0.0,490.0,1145.0,1145,1053,0,2198,836.0,192,84,0,0,0,0,0,3.0,SBrkr,8,5,Gd,TA,Gd,TA,Av,GLQ,Ex,Gd,TA,Typ,250000.0


In [57]:
#전처리1 - 연속형데이터 중 null값이 있는 행 모두 버려주기
new_total[number_selected].isnull().sum()

LotFrontage      486
LotArea            0
YearBuilt          0
YearRemodAdd       0
MasVnrArea        23
BsmtFinSF1         1
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
GarageArea         1
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
dtype: int64

In [58]:
new_total["YearBuilt"] = new_total["YearBuilt"].fillna(new_total.YearBuilt.mean())

In [59]:
new_total["BsmtFinSF1"] = new_total["BsmtFinSF1"].fillna(new_total.BsmtFinSF1.mean())

In [60]:
new_total["BsmtUnfSF"] = new_total["BsmtUnfSF"].fillna(new_total.BsmtUnfSF.mean())

In [61]:
new_total["TotalBsmtSF"] = new_total["TotalBsmtSF"].fillna(new_total.TotalBsmtSF.mean())

In [62]:
new_total["GarageArea"] = new_total["GarageArea"].fillna(new_total.GarageArea.mean())

In [63]:
new_total["GarageYrBlt"] = new_total["GarageYrBlt"].fillna(new_total.GarageYrBlt.mean())

In [64]:
new_total["Electrical"] = new_total["Electrical"].fillna("SBrkr")

In [65]:
new_total = new_total.fillna("404")

In [66]:
new_total.isnull().sum()

Alley            0
BedroomAbvGr     0
BldgType         0
BsmtFinType2     0
BsmtFullBath     0
BsmtHalfBath     0
CentralAir       0
Condition1       0
Condition2       0
Exterior1st      0
Exterior2nd      0
Fence            0
Fireplaces       0
Foundation       0
FullBath         0
GarageCond       0
GarageFinish     0
GarageQual       0
GarageType       0
GarageYrBlt      0
HalfBath         0
Heating          0
HouseStyle       0
Id               0
KitchenAbvGr     0
LandContour      0
LandSlope        0
LotConfig        0
LotShape         0
MSSubClass       0
                ..
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
GarageCars       0
Electrical       0
OverallQual      0
OverallCond      0
ExterQual        0
ExterCond        0
BsmtQual    

In [67]:
new_total.head()

,Alley,BedroomAbvGr,BldgType,BsmtFinType2,BsmtFullBath,BsmtHalfBath,CentralAir,Condition1,Condition2,Exterior1st,Exterior2nd,Fence,Fireplaces,Foundation,FullBath,GarageCond,GarageFinish,GarageQual,GarageType,GarageYrBlt,HalfBath,Heating,HouseStyle,Id,KitchenAbvGr,LandContour,LandSlope,LotConfig,LotShape,MSSubClass,MSZoning,MasVnrType,MiscFeature,MoSold,Neighborhood,PavedDrive,PoolQC,RoofMatl,RoofStyle,SaleCondition,SaleType,Street,TotRmsAbvGrd,Utilities,YrSold,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,GarageCars,Electrical,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,KitchenQual,FireplaceQu,Functional,SalePrice
0,404,3,1Fam,Unf,1,0,Y,Norm,Norm,VinylSd,VinylSd,404,0,PConc,2,TA,RFn,TA,Attchd,2003.0,1,GasA,2Story,1,1,Lvl,Gtl,Inside,Reg,60,RL,BrkFace,404,2,CollgCr,Y,404,CompShg,Gable,Normal,WD,Pave,8,AllPub,2008,65,8450,2003,2003,196,706.0,0,150.0,856.0,856,854,0,1710,548.0,0,61,0,0,0,0,0,2,SBrkr,7,5,Gd,TA,Gd,TA,No,GLQ,Ex,Gd,404,Typ,208500
1,404,3,1Fam,Unf,0,1,Y,Feedr,Norm,MetalSd,MetalSd,404,1,CBlock,2,TA,RFn,TA,Attchd,1976.0,0,GasA,1Story,2,1,Lvl,Gtl,FR2,Reg,20,RL,None,404,5,Veenker,Y,404,CompShg,Gable,Normal,WD,Pave,6,AllPub,2007,80,9600,1976,1976,0,978.0,0,284.0,1262.0,1262,0,0,1262,460.0,298,0,0,0,0,0,0,2,SBrkr,6,8,TA,TA,Gd,TA,Gd,ALQ,Ex,TA,TA,Typ,181500
2,404,3,1Fam,Unf,1,0,Y,Norm,Norm,VinylSd,VinylSd,404,1,PConc,2,TA,RFn,TA,Attchd,2001.0,1,GasA,2Story,3,1,Lvl,Gtl,Inside,IR1,60,RL,BrkFace,404,9,CollgCr,Y,404,CompShg,Gable,Normal,WD,Pave,6,AllPub,2008,68,11250,2001,2002,162,486.0,0,434.0,920.0,920,866,0,1786,608.0,0,42,0,0,0,0,0,2,SBrkr,7,5,Gd,TA,Gd,TA,Mn,GLQ,Ex,Gd,TA,Typ,223500
3,404,3,1Fam,Unf,1,0,Y,Norm,Norm,Wd Sdng,Wd Shng,404,1,BrkTil,1,TA,Unf,TA,Detchd,1998.0,0,GasA,2Story,4,1,Lvl,Gtl,Corner,IR1,70,RL,None,404,2,Crawfor,Y,404,CompShg,Gable,Abnorml,WD,Pave,7,AllPub,2006,60,9550,1915,1970,0,216.0,0,540.0,756.0,961,756,0,1717,642.0,0,35,272,0,0,0,0,3,SBrkr,7,5,TA,TA,TA,Gd,No,ALQ,Gd,Gd,Gd,Typ,140000
4,404,4,1Fam,Unf,1,0,Y,Norm,Norm,VinylSd,VinylSd,404,1,PConc,2,TA,RFn,TA,Attchd,2000.0,1,GasA,2Story,5,1,Lvl,Gtl,FR2,IR1,60,RL,BrkFace,404,12,NoRidge,Y,404,CompShg,Gable,Normal,WD,Pave,9,AllPub,2008,84,14260,2000,2000,350,655.0,0,490.0,1145.0,1145,1053,0,2198,836.0,192,84,0,0,0,0,0,3,SBrkr,8,5,Gd,TA,Gd,TA,Av,GLQ,Ex,Gd,TA,Typ,250000


In [68]:
#명목형 변수를 연속형으로 바꿔주는 것

In [69]:
new_total = num_category(new_total, "ExterQual", ["Po", "Fa", "TA", "Gd", "Ex"])

In [70]:
new_total = num_category(new_total, "ExterCond", ["Po", "Fa", "TA", "Gd", "Ex"])

In [71]:
new_total = num_category(new_total, "BsmtQual", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [72]:
new_total = num_category(new_total, "BsmtCond", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [73]:
new_total = num_category(new_total, "BsmtExposure", ["404", "No", "Fa", "Mn", "Av", "Gd"])

In [74]:
new_total = num_category(new_total, "BsmtFinType1", ["404", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"])

In [75]:
new_total = num_category(new_total, "HeatingQC", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [76]:
new_total = num_category(new_total, "KitchenQual", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [77]:
new_total = num_category(new_total, "FireplaceQu", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [78]:
new_total = num_category(new_total, "Electrical", ["Mix", "FuseP", "FuseF", "FuseA", "SBrkr"])

In [79]:
new_total = num_category(new_total, "Functional", ["404", "Sal", "Sev", "Maj2", "Maj1", "Mod", "Min2", "Min1", "Typ"])
new_total.Functional.unique()

array([8, 7, 4, 6, 5, 3, 2, 0], dtype=int64)

In [80]:
#변수끼리 합쳐주는 것

In [81]:
new_total["is_Functional"] = total["Functional"].map(lambda x : 1 if x==8 else 0)
new_total["is_LotConfig"] = new_total.LotConfig.map(lambda x : 1 if x == "CulDSac" else 0)
new_total["is_FullBath"] = new_total.FullBath.map(lambda x : 1 if x <=2 else x-1)
new_total["is_OverallCond"] = new_total.OverallCond.map(lambda x : 1 if x <=4 else 2)
new_total["is_BldgType"] = new_total.BldgType.map(lambda x : 1 if x =="1Fam" else 2)
#new_total = number_group_cols(new_total, ["TotRmsAbvGrd"], [13])
#new_total = number_group_cols(new_total, ["HalfBath"], [1])
#new_total = number_group_cols(new_total, ["FullBath"], [2])

In [82]:
new_total["is_highQU"] = new_total.OverallQual.map(lambda x : 1 if x>=10 else 0)

In [83]:
#임시 전처리

In [84]:
new_total = new_total.assign(Is_Remod =  lambda x : x["YearBuilt"] != x["YearRemodAdd"])
new_total["Is_Remod"] = new_total.YearRemodAdd.map(lambda x : 1 if x == True else 0)
new_total["after"] = 2011 - new_total["YearRemodAdd"]
new_total = new_total.drop("YearRemodAdd", axis = 1)

In [85]:
#new_total = new_total.assign(Is_2nd =  lambda x : x["2ndFlrSF"] != 0)
#new_total["Is_2nd"] = new_total.Is_2nd.map(lambda x : 1 if x == True else 0)

In [86]:
#new_total = new_total.assign(Is_bsmt_unfinish = lambda x : x["BsmtFinType1"] == 0)
#new_total["Is_bsmt_unfinish"] = new_total.Is_bsmt_unfinish.map(lambda x : 1 if x == True else 0)

In [87]:
#new_total = new_total.assign(Is_Fireplaces = lambda x : x["Fireplaces"] != 0)
#new_total["Is_Fireplaces"] = new_total.Is_Fireplaces.map(lambda x : 1 if x == True else 0)

In [88]:
#new_total = new_total.assign(Is_GarageFinish = lambda x : x["GarageFinish"] != "Unf")
#new_total["Is_GarageFinish "] = new_total.Is_GarageFinish .map(lambda x : 1 if x == True else 0)

In [89]:
new_total["total_qual"] = (new_total['OverallQual'] + new_total["ExterQual"]+new_total["BsmtQual"]+ new_total["HeatingQC"] + 
                           new_total["KitchenQual"] + new_total["FireplaceQu"])

In [90]:
new_total["total_point"] = (new_total['OverallQual'] + new_total["ExterQual"]+
                            new_total["BsmtQual"]+ new_total["Functional"] + 
                            new_total["Electrical"]+new_total["ExterCond"] +
                            new_total["BsmtCond"] + new_total["OverallCond"] +
                            new_total["BsmtExposure"] + new_total["HeatingQC"] + new_total["KitchenQual"] + new_total["FireplaceQu"] +
                            new_total["BsmtFinType1"])

In [91]:
new_total["total_cond"] = (new_total["ExterCond"] +
                            new_total["BsmtCond"] + new_total["OverallCond"] )

In [92]:
new_total["ratio_bsmt"] = new_total["BsmtUnfSF"] / new_total["TotalBsmtSF"]
new_total["ratio_bsmt"] = new_total["ratio_bsmt"].map(lambda x : x if x>=0 else 100000)

In [93]:
new_total["Total_Area"] = new_total["GrLivArea"] + new_total["LotArea"] + new_total["GarageArea"] + new_total["TotalBsmtSF"]

In [94]:
new_total["GrLivArea"] = new_total["GrLivArea"].map(lambda x : math.log(x))

In [95]:
new_total["LotArea"] = new_total["LotArea"].map(lambda x : math.log(x))

In [96]:
new_total["GarageArea"] = new_total["GarageArea"].map(lambda x : math.log(x+1))

In [97]:
new_total["TotalBsmtSF"] = new_total["TotalBsmtSF"].map(lambda x : math.log(x+1))

In [98]:
new_total["Total_Area"] = new_total["Total_Area"].map(lambda x : math.log(x+1))

In [99]:
new_total["BsmtFinSF1"] = new_total["BsmtFinSF1"].map(lambda x : math.log(x+1))

In [100]:
new_total["BsmtUnfSF"] = new_total["BsmtUnfSF"].map(lambda x : math.log(x+1))

In [101]:
new_total["1stFlrSF"] = new_total["1stFlrSF"].map(lambda x : math.log(x+1))

In [102]:
new_total["2ndFlrSF"] = new_total["2ndFlrSF"].map(lambda x : math.log(x+1))

In [103]:
new_total["MasVnrArea"] = new_total["MasVnrArea"].map(lambda x : math.log(int(x)+1))
new_total["WoodDeckSF"] = new_total["WoodDeckSF"].map(lambda x : math.log(int(x)+1))
new_total["OpenPorchSF"] = new_total["OpenPorchSF"].map(lambda x : math.log(int(x)+1))

In [104]:
new_total["SalePrice"] = new_total.SalePrice.map(lambda x: math.log(int(x)+1))

In [105]:
#확인

In [106]:
new_total.head(2)

,Alley,BedroomAbvGr,BldgType,BsmtFinType2,BsmtFullBath,BsmtHalfBath,CentralAir,Condition1,Condition2,Exterior1st,Exterior2nd,Fence,Fireplaces,Foundation,FullBath,GarageCond,GarageFinish,GarageQual,GarageType,GarageYrBlt,HalfBath,Heating,HouseStyle,Id,KitchenAbvGr,LandContour,LandSlope,LotConfig,LotShape,MSSubClass,MSZoning,MasVnrType,MiscFeature,MoSold,Neighborhood,PavedDrive,PoolQC,RoofMatl,RoofStyle,SaleCondition,SaleType,Street,TotRmsAbvGrd,Utilities,YrSold,LotFrontage,LotArea,YearBuilt,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,GarageCars,Electrical,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,KitchenQual,FireplaceQu,Functional,SalePrice,is_Functional,is_LotConfig,is_FullBath,is_OverallCond,is_BldgType,is_highQU,Is_Remod,after,total_qual,total_point,total_cond,ratio_bsmt,Total_Area
0,404,3,1Fam,Unf,1,0,Y,Norm,Norm,VinylSd,VinylSd,404,0,PConc,2,TA,RFn,TA,Attchd,2003.0,1,GasA,2Story,1,1,Lvl,Gtl,Inside,Reg,60,RL,BrkFace,404,2,CollgCr,Y,404,CompShg,Gable,Normal,WD,Pave,8,AllPub,2008,65,9.041922,2003,5.283204,6.561031,0,5.017280,6.753438,6.753438,6.751101,0,7.444249,6.308098,0.000000,4.127134,0,0,0,0,0,2,4,7,5,3,2,4,3,1,6,5,4,0,8,12.247699,0,0,1,2,1,0,0,8,23,52,10,0.175234,9.355739
1,404,3,1Fam,Unf,0,1,Y,Feedr,Norm,MetalSd,MetalSd,404,1,CBlock,2,TA,RFn,TA,Attchd,1976.0,0,GasA,1Story,2,1,Lvl,Gtl,FR2,Reg,20,RL,None,404,5,Veenker,Y,404,CompShg,Gable,Normal,WD,Pave,6,AllPub,2007,80,9.169518,1976,0.000000,6.886532,0,5.652489,7.141245,7.141245,0.000000,0,7.140453,6.133398,5.700444,0.000000,0,0,0,0,0,2,4,6,8,2,2,4,3,5,5,5,3,3,8,12.109016,0,0,1,2,1,0,0,35,23,58,13,0.225040,9.440261


In [107]:
#히트맵으로 확인
#training = training = new_total.iloc[:len(training)]

In [108]:
#n= len(selected_with_SalePrice)
#colormap = plt.cm.viridis
#plt.figure(figsize=(n,n))
#plt.title('Pearson Correlation of Features', y=1.05, size=15)
#sns.heatmap(training.corr(),
#            linewidths=0.1,
#            vmax=1.0, 
#            square=True, 
#            cmap=colormap, 
#            linecolor='white', 
#            annot=True)

In [109]:
new_total2 = dummify_category_cols(new_total,category_selected)

In [110]:
new_total2.head(2)

,LotFrontage,LotArea,YearBuilt,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,GarageCars,Electrical,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,KitchenQual,FireplaceQu,Functional,SalePrice,is_Functional,is_LotConfig,is_FullBath,is_OverallCond,is_BldgType,is_highQU,Is_Remod,after,total_qual,total_point,total_cond,ratio_bsmt,Total_Area,Alley_Grvl,Alley_Pave,BedroomAbvGr_0,BedroomAbvGr_1,BedroomAbvGr_2,BedroomAbvGr_3,BedroomAbvGr_4,BedroomAbvGr_5,BedroomAbvGr_6,BedroomAbvGr_8,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,BsmtFullBath_0.0,BsmtFullBath_1.0,BsmtFullBath_2.0,BsmtFullBath_3.0,BsmtHalfBath_0.0,BsmtHalfBath_1.0,BsmtHalfBath_2.0,CentralAir_N,CentralAir_Y,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,...,MasVnrType_None,MasVnrType_Stone,MiscFeature_Gar2,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MoSold_1,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Street_Grvl,Street_Pave,TotRmsAbvGrd_2,TotRmsAbvGrd_3,TotRmsAbvGrd_4,TotRmsAbvGrd_5,TotRmsAbvGrd_6,TotRmsAbvGrd_7,TotRmsAbvGrd_8,TotRmsAbvGrd_9,TotRmsAbvGrd_10,TotRmsAbvGrd_11,TotRmsAbvGrd_12,TotRmsAbvGrd_13,TotRmsAbvGrd_14,TotRmsAbvGrd_15,Utilities_AllPub,Utilities_NoSeWa,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010
0,65,9.041922,2003,5.283204,6.561031,0,5.017280,6.753438,6.753438,6.751101,0,7.444249,6.308098,0.000000,4.127134,0,0,0,0,0,2,4,7,5,3,2,4,3,1,6,5,4,0,8,12.247699,0,0,1,2,1,0,0,8,23,52,10,0.175234,9.355739,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0
1,80,9.169518,1976,0.000000,6.886532,0,5.652489,7.141245,7.141245,0.000000,0,7.140453,6.133398,5.700444,0.000000,0,0,0,0,0,2,4,6,8,2,2,4,3,5,5,5,3,3,8,12.109016,0,0,1,2,1,0,0,35,23,58,13,0.225040,9.440261,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0


In [111]:
training = new_total2.iloc[:len(training)]
test = new_total2.iloc[len(training):]

In [112]:
#여기서 selected 재설정

In [113]:
new_total.isnull().sum()

Alley             0
BedroomAbvGr      0
BldgType          0
BsmtFinType2      0
BsmtFullBath      0
BsmtHalfBath      0
CentralAir        0
Condition1        0
Condition2        0
Exterior1st       0
Exterior2nd       0
Fence             0
Fireplaces        0
Foundation        0
FullBath          0
GarageCond        0
GarageFinish      0
GarageQual        0
GarageType        0
GarageYrBlt       0
HalfBath          0
Heating           0
HouseStyle        0
Id                0
KitchenAbvGr      0
LandContour       0
LandSlope         0
LotConfig         0
LotShape          0
MSSubClass        0
                 ..
PoolArea          0
MiscVal           0
GarageCars        0
Electrical        0
OverallQual       0
OverallCond       0
ExterQual         0
ExterCond         0
BsmtQual          0
BsmtCond          0
BsmtExposure      0
BsmtFinType1      0
HeatingQC         0
KitchenQual       0
FireplaceQu       0
Functional        0
SalePrice         0
is_Functional     0
is_LotConfig      0


In [114]:
training['is_train'] = np.random.uniform(0, 1, len(training)) <= .75

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [115]:
training_set = training[training['is_train']==True]
test_set = training[training['is_train']==False]

In [116]:
forest = RandomForestRegressor(n_estimators=6000, n_jobs=3)
#adaboost = AdaBoostRegressor(n_estimators=5000)
#gradientboosting = GradientBoostingRegressor(loss = "huber", n_estimators=5000, max_depth=15)
#extratrees = ExtraTreesRegressor(n_estimators=6000, n_jobs =3)
bagging = BaggingRegressor(n_estimators=6000,n_jobs =3 )
#mlp = MLPRegressor(hidden_layer_sizes = (150,1), activation = "relu", max_iter=100000)

In [117]:
feature = new_total2.columns.drop("SalePrice")
target = ["SalePrice"]

In [925]:
forest.fit(training_set[feature], training_set[target])
#adaboost.fit(training_set[feature], training_set[target])
#gradientboosting.fit(training_set[feature], training_set[target])
#extratrees.fit(training_set[feature], training_set[target])
bagging.fit(training_set[feature], training_set[target])
#mlp.fit(training_set[feature], training_set[target])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



BaggingRegressor(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=6000, n_jobs=3, oob_score=False, random_state=None,
         verbose=0, warm_start=False)

In [926]:
#forest.predict(test_set[feature])

In [927]:
print("forest : ",forest.score(test_set[feature], test_set[target]))
#print("adaboost : ",adaboost.score(test_set[feature], test_set[target]))
#print("gb : ",gradientboosting.score(test_set[feature], test_set[target]))
#print("extratrees : ",extratrees.score(test_set[feature], test_set[target]))
print("bagging : ",bagging.score(test_set[feature], test_set[target]))
#print("mlp : ",mlp.score(test_set[feature], test_set[target]))

forest :  0.868241034779
bagging :  0.86859496568


In [928]:
y_pred1=forest.predict(test_set[feature])
#y_pred2=adaboost.predict(test_set[feature])
#y_pred3=gradientboosting.predict(test_set[feature])
#y_pred4=extratrees.predict(test_set[feature])
y_pred5=bagging.predict(test_set[feature])
#y_pred6=mlp.predict(test_set[feature])

In [929]:
y_pred1 = np.array(list(map(lambda x: math.exp(x)-1,y_pred1)))
#y_pred2 = np.array(list(map(lambda x: math.exp(x)-1,y_pred2)))
#y_pred3 = np.array(list(map(lambda x: math.exp(x)-1,y_pred3)))
#y_pred4 = np.array(list(map(lambda x: math.exp(x)-1,y_pred4)))
y_pred5 = np.array(list(map(lambda x: math.exp(x)-1,y_pred5)))
#y_pred6 = np.array(list(map(lambda x: math.exp(x)-1,y_pred6)))

In [930]:
y_true = test_set["SalePrice"]
y_ture= np.array(list(map(lambda x: math.exp(x)-1,y_true)))

In [931]:
rmse1 = RMSE(np.exp(y_true),y_pred1)
#rmse2 = RMSE(np.exp(y_true),y_pred2)
#rmse3 = RMSE(np.exp(y_true),y_pred3)
#rmse4 = RMSE(np.exp(y_true),y_pred4)
rmse5 = RMSE(np.exp(y_true),y_pred5)
#rmse6 = RMSE(np.exp(y_true),y_pred6)
print("forest : ",rmse1)
#print("adaboost : ",rmse2)
#print("gb : ",rmse3)
#print("extratrees : ",rmse4)
print("bagging : ",rmse5)
#print("mlp : ",rmse6)

forest :  29763.288015306334
bagging :  29660.640224598763


In [ ]:
#importance = pd.DataFrame(forest.feature_importances_, index = training_set[feature].columns)

In [ ]:
#(
#    importance.assign(header = lambda x: x.index.str.split("_").str[0])
#                  .groupby("header")[0]
#                  .sum()
#                  .iplot(kind="bar")
#)

In [ ]:
#최적화

In [ ]:
best_score = 0
for N in range(4000,8500,500):
    tmp = RandomForestRegressor(n_estimators=N)
    tmp.fit(training_set[feature], training_set[target])
    score = tmp.score(test_set[feature],test_set[target])
    print("최적화 n_estimator 값 : {}".format(N))
    print("예측률 : {}".format(score))
    
    if score>best_score:
        best_score = score
        best_N = N
        best_rf_model = tmp
        
print("최적화 n_estimator 값 : {}".format(best_N))
print("예측률 : {}".format(best_score))

In [ ]:
best_score = 0
for N in range(13,21):
    tmp = RandomForestRegressor(n_estimators=4500, max_depth = N)
    tmp.fit(training_set[feature], training_set[target])
    score = tmp.score(test_set[feature],test_set[target])
    
    if score>best_score:
        best_score = score
        best_N = N
        best_rf_model = tmp
        
print("최적화 max_depth 값 : {}".format(best_N))
print("예측률 : {}".format(best_score))

In [933]:
#제출

In [118]:
forest.fit(training[feature], training[target])
#adaboost.fit(training[feature], training[target])
#gradientboosting.fit(training[feature], training[target])
#extratrees.fit(training[feature], training[target])
bagging.fit(training[feature], training[target])
#mlp.fit(training[feature], training[target])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



BaggingRegressor(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=6000, n_jobs=3, oob_score=False, random_state=None,
         verbose=0, warm_start=False)

In [119]:
prediction1 =forest.predict(test[feature])
#prediction2 =adaboost.predict(test[feature])
#prediction3 =gradientboosting.predict(test[feature])
#prediction4 =extratrees.predict(test[feature])
prediction5 =bagging.predict(test[feature])
#prediction6 =mlp.predict(test[feature])

In [120]:
prediction1 = np.array(list(map(lambda x: math.exp(x)-1,prediction1)))
#prediction2 = np.array(list(map(lambda x: math.exp(x)-1,prediction2)))
#prediction3 = np.array(list(map(lambda x: math.exp(x)-1,prediction3)))
#prediction4 = np.array(list(map(lambda x: math.exp(x)-1,prediction4)))
prediction5 = np.array(list(map(lambda x: math.exp(x)-1,prediction5)))
#prediction6 = np.array(list(map(lambda x: math.exp(x)-1,prediction6)))

In [121]:
submission1 = pd.DataFrame({'Id':list(test.index+1),'SalePrice':prediction1})
#submission2 = pd.DataFrame({'Id':list(test.index+1),'SalePrice':prediction2})
#submission3 = pd.DataFrame({'Id':list(test.index+1),'SalePrice':prediction3})
#submission4 = pd.DataFrame({'Id':list(test.index+1),'SalePrice':prediction4})
submission5 = pd.DataFrame({'Id':list(test.index+1),'SalePrice':prediction5})
#submission6 = pd.DataFrame({'Id':list(test.index+1),'SalePrice':prediction6})

In [122]:
submission1.to_csv("Submission_forest6.csv")
#submission2.to_csv("Submission_adaboost3.csv")
#submission3.to_csv("Submission_gradientboosting3.csv")
#submission4.to_csv("Submission_extratrees3.csv")
submission5.to_csv("Submission_bagging6.csv")
#submission6.to_csv("Submission_mlp1.csv")